In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib as mpl
%matplotlib inline

In [15]:
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# 读取事件资料

In [16]:
df_event = pd.read_excel('MS_event17.xlsx')
df_event['M_group'] = [ str(x)[4:6] for x in df_event.Date ]
df_event['M_group_desc'] = [ str(x)[4:6] for x in df_event.Date_desclosure ]
Event_list = df_event['Event.1'].unique()

In [17]:
df_MS_PandQ=pd.read_csv('MS_PandQ.csv')

In [18]:
#匯入產業類別資料
df_IND = pd.read_excel('MS_IND.xlsx')
df_IND.columns = ['Code','Name','CSRC_IND_Code','Wind_IND_Code']
len(df_IND['Wind_IND_Code'].unique())

23

In [19]:
IND_list = df_IND['Wind_IND_Code'].unique()
#      ['Wind制药、生物科技与生命科学', 'Wind材料Ⅱ', 'Wind耐用消费品与服装', 'Wind半导体与半导体生产设备',
#       'Wind资本货物', 'Wind技术硬件与设备', 'Wind房地产Ⅱ', 'Wind能源Ⅱ', 'Wind零售业',
#       'Wind媒体Ⅱ', 'Wind消费者服务Ⅱ', 'Wind商业和专业服务', 'Wind公用事业Ⅱ', 'Wind运输',
#       'Wind食品、饮料与烟草', 'Wind医疗保健设备与服务', 'Wind汽车与汽车零部件', 'Wind软件与服务',
#       'Wind家庭与个人用品', 'Wind银行', 'Wind食品与主要用品零售Ⅱ', 'Wind多元金融', 'Wind电信服务Ⅱ']
for i in range(0,len(IND_list)):
    df_IND['IND_dummy_'+str(i)] = ( df_IND['Wind_IND_Code'] == IND_list[i])*1 #dummy 1 是制药、生物科技与生命科学

In [20]:
df_MS_PQ_Ind = pd.merge(left=df_MS_PandQ,right=df_IND,on=['Code'],how='left')

In [21]:
len(df_MS_PQ_Ind) == len(df_MS_PandQ)

True

# 創造市值dummy 共5類

In [22]:
df_Cap =pd.read_csv('Cap_data.csv')

In [23]:

df_MS_PQ_Ind_Cap = pd.merge(left = df_MS_PQ_Ind,right=df_Cap[['Date','Code','Tournover','Cap']],on=['Date','Code'])

In [24]:
len(df_MS_PQ_Ind_Cap) == len(df_MS_PQ_Ind)

True

In [25]:

first_Quintile = df_MS_PQ_Ind_Cap.groupby('Date')['Cap'].apply(lambda x : np.where(np.array(x) < np.percentile(x,20),1,0))
second_Quintile = df_MS_PQ_Ind_Cap.groupby('Date')['Cap'].apply( lambda x :np.where((np.array(x) >= np.percentile(x,20))&
                                                                               (np.array(x) <  np.percentile(x,40))
                                                                              ,1,0) )
third_Quintile = df_MS_PQ_Ind_Cap.groupby('Date')['Cap'].apply( lambda x :np.where((np.array(x) >= np.percentile(x,40))&
                                                                              (np.array(x) <  np.percentile(x,60))
                                                                              ,1,0) )
fourth_Quintile = df_MS_PQ_Ind_Cap.groupby('Date')['Cap'].apply( lambda x :np.where((np.array(x) >= np.percentile(x,60))&
                                                                               (np.array(x) <  np.percentile(x,80))
                                                                              ,1,0) )
fifth_Quintile = df_MS_PQ_Ind_Cap.groupby('Date')['Cap'].apply(lambda x : np.where(np.array(x) >= np.percentile(x,80),1,0))

In [26]:
#由於上方是GroupbyDate 產出結果依照時間順序往下牌
df_sortdate = df_MS_PQ_Ind_Cap.sort_values(['Date','Code'],ascending=[1,1]).reset_index(drop=True)
df_sortdate['First_Quintile'] = [ y for x in first_Quintile for y in x]
df_sortdate['Second_Quintile'] = [ y for x in second_Quintile for y in x]
df_sortdate['Third_Quintile'] = [ y for x in third_Quintile for y in x]
df_sortdate['Fourth_Quintile'] = [ y for x in fourth_Quintile for y in x]
df_sortdate['Fifth_Quintile'] = [ y for x in fifth_Quintile for y in x]
df_sortdate['Quintile_Class'] = df_sortdate['First_Quintile'].values*1+df_sortdate['Second_Quintile'].values*2+df_sortdate['Third_Quintile'].values*3+df_sortdate['Fourth_Quintile'].values*4+df_sortdate['Fifth_Quintile'].values*5

In [27]:
print(df_sortdate.groupby('Date')['Code'].count())
print(df_sortdate.groupby('Date')['First_Quintile'].sum())
print(df_sortdate.groupby('Date')['Second_Quintile'].sum())
print(df_sortdate.groupby('Date')['Third_Quintile'].sum())
print(df_sortdate.groupby('Date')['Fourth_Quintile'].sum())
print(df_sortdate.groupby('Date')['Fifth_Quintile'].sum())#確定沒算錯

Date
20130902    668
20130903    668
20130904    670
20130905    671
20130906    667
20130909    666
20130910    665
20130911    666
20130912    667
20130913    667
20130916    667
20130917    667
20130918    667
20130923    666
20130924    669
20130925    667
20130926    665
20130927    667
20130930    668
20131008    666
20131009    663
20131010    666
20131011    667
20131014    664
20131015    665
20131016    662
20131017    662
20131018    663
20131021    662
20131022    663
           ... 
20180613    843
20180614    841
20180615    840
20180619    833
20180620    833
20180621    833
20180622    834
20180625    837
20180626    839
20180627    841
20180628    843
20180629    844
20180702    847
20180703    852
20180704    852
20180705    852
20180706    855
20180709    857
20180710    858
20180711    858
20180712    858
20180713    859
20180716    859
20180717    859
20180718    859
20180719    859
20180720    861
20180723    861
20180724    861
20180727    911
Name: Code, Length:

In [28]:
df_MS_PQ_Ind_Cap = df_sortdate.sort_values(['Code','Date'],ascending=[1,1]).reset_index(drop=True)

In [29]:
#取前后五天分析
for i in range(-30,11):
    df_MS_PQ_Ind_Cap['Volume_t'+str(i)] = df_MS_PQ_Ind_Cap.groupby('Code')['Volume'].shift(-i)
#取T+1开盘价分析
df_MS_PQ_Ind_Cap['Open_t+1'] = df_MS_PQ_Ind_Cap.groupby('Code')['Open'].shift(-1)

In [32]:
df_MS_PQ_Ind_Cap['MeanPast30V'] = np.mean(df_MS_PQ_Ind_Cap[['Volume_t-30',
       'Volume_t-29', 'Volume_t-28', 'Volume_t-27', 'Volume_t-26',
       'Volume_t-25', 'Volume_t-24', 'Volume_t-23', 'Volume_t-22',
       'Volume_t-21', 'Volume_t-20', 'Volume_t-19', 'Volume_t-18',
       'Volume_t-17', 'Volume_t-16', 'Volume_t-15', 'Volume_t-14',
       'Volume_t-13', 'Volume_t-12', 'Volume_t-11', 'Volume_t-10',
       'Volume_t-9', 'Volume_t-8', 'Volume_t-7', 'Volume_t-6', 'Volume_t-5',
       'Volume_t-4', 'Volume_t-3', 'Volume_t-2', 'Volume_t-1']],axis=1)
df_MS_PQ_Ind_Cap['StdPast30V'] = np.std(df_MS_PQ_Ind_Cap[['Volume_t-30',
       'Volume_t-29', 'Volume_t-28', 'Volume_t-27', 'Volume_t-26',
       'Volume_t-25', 'Volume_t-24', 'Volume_t-23', 'Volume_t-22',
       'Volume_t-21', 'Volume_t-20', 'Volume_t-19', 'Volume_t-18',
       'Volume_t-17', 'Volume_t-16', 'Volume_t-15', 'Volume_t-14',
       'Volume_t-13', 'Volume_t-12', 'Volume_t-11', 'Volume_t-10', 
       'Volume_t-9', 'Volume_t-8', 'Volume_t-7', 'Volume_t-6', 'Volume_t-5',
       'Volume_t-4', 'Volume_t-3', 'Volume_t-2', 'Volume_t-1']],axis=1)
df_MS_PQ_Ind_Cap['UpperBound'] = df_MS_PQ_Ind_Cap['MeanPast30V'].values + df_MS_PQ_Ind_Cap['StdPast30V'].values
df_MS_PQ_Ind_Cap['LowerBound'] = df_MS_PQ_Ind_Cap['MeanPast30V'].values - df_MS_PQ_Ind_Cap['StdPast30V'].values
df_MS_PQ_Ind_Cap['HighThan1Std'] = np.where((df_MS_PQ_Ind_Cap['Volume'].values-df_MS_PQ_Ind_Cap['UpperBound'].values)>0,1,0)
df_MS_PQ_Ind_Cap['LowThan1Std'] = np.where((df_MS_PQ_Ind_Cap['Volume'].values-df_MS_PQ_Ind_Cap['LowerBound'].values)<0,1,0)


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in greater
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in less


In [33]:
#筛除缺少资料其间 与对其事件与价格量资料 5日
df_MS_PQ_Ind_Cap=df_MS_PQ_Ind_Cap[(df_MS_PQ_Ind_Cap.Date.astype('int') > 20140720)&(df_MS_PQ_Ind_Cap.Date.astype('int') < 20180717)].reset_index(drop=True)
df_event=df_event[df_event.Date.astype('int')<20180710].reset_index(drop=True)

In [34]:
print(len(df_MS_PQ_Ind_Cap))
print(len(df_event))

705088
108238


# 將事件資料併入股價量價資料 left 量價 right 事件

In [35]:
print(len(df_event.drop_duplicates().reset_index(drop=True)))
print(len(df_event))#這兩個數有差異 表示有重複的事件資料
df_event_dropdu = df_event.drop_duplicates().reset_index(drop=True)

108026
108238


In [36]:
df_event_dropdu.Date = df_event_dropdu.Date.astype('str')
df_MS_PQ_Ind_Cap.Date = df_MS_PQ_Ind_Cap.Date.astype('str')
df_event_dropdu.Code = df_event_dropdu.Code.astype('str')
df_MS_PQ_Ind_Cap.Code = df_MS_PQ_Ind_Cap.Code.astype('str')

df_PandQ_Event = pd.merge(left=df_MS_PQ_Ind_Cap,right=df_event_dropdu ,on=['Date','Code'],how='left')

In [37]:
df_PandQ_Event[((df_PandQ_Event.Date == '20140729') & (df_PandQ_Event.Code == '002306.SZ') )] 

,Unnamed: 0,Date,Code,Open,High,Low,Close,Volume,Amount,Name_x,...,UpperBound,LowerBound,HighThan1Std,LowThan1Std,Name_y,Event,Event.1,Date_desclosure,M_group,M_group_desc
264432,326977,20140729,002306.SZ,6.13,6.13,6.03,6.13,23610421.0,144662896.0,*ST云网,...,3.176678e+07,876194.118012,0,0,*ST云网,交易,复牌,NaN,07,
264433,326977,20140729,002306.SZ,6.13,6.13,6.03,6.13,23610421.0,144662896.0,*ST云网,...,3.176678e+07,876194.118012,0,0,*ST云网,交易,大宗交易,NaN,07,
264434,326977,20140729,002306.SZ,6.13,6.13,6.03,6.13,23610421.0,144662896.0,*ST云网,...,3.176678e+07,876194.118012,0,0,*ST云网,股權質押,股權質押公告,20140729.0,07,07


In [38]:
#测试有无为0得序列
df_testzero = df_PandQ_Event[['Volume_t-5', 'Volume_t-4', 'Volume_t-3', 'Volume_t-2',
       'Volume_t-1', 'Volume_t0', 'Volume_t1', 'Volume_t2', 'Volume_t3',
       'Volume_t4', 'Volume_t5', 'Open_t+1']]
for i in range(0,len(df_testzero.columns)):
    print(sum((df_testzero.iloc[:,i] == 0)*1)) #很好都没有0的 下面就不会有无限大的可能安心算變動率

0
0
0
0
0
0
49
50
50
52
53
0


In [39]:
df_PandQ_Event['Chg_t~t+1%'] =( df_PandQ_Event['Volume'].values - df_PandQ_Event['Volume_t-1'].values)/df_PandQ_Event['Volume_t-1'].values
df_PandQ_Event['E_dummy1'] = (df_PandQ_Event['Event.1'] == '大宗交易')*1
df_PandQ_Event['E_dummy2'] = (df_PandQ_Event['Event.1'] == '股權質押公告')*1
df_PandQ_Event['E_dummy3'] = (df_PandQ_Event['Event.1'] == '撤消ST')*1
df_PandQ_Event['E_dummy4'] = (df_PandQ_Event['Event.1'] == '纳入重要指数')*1

In [40]:
print(len(df_MS_PQ_Ind_Cap))
df_PandQ_Event_dropMultiEvent = df_PandQ_Event[['Code','Date', 'Close','Volume','HighThan1Std','LowThan1Std',
                                                'Chg_t~t+1%','Open_t+1','High','Low','Wind_IND_Code','Volume_t-5', 'Volume_t-4', 'Volume_t-3', 'Volume_t-2',
                                                'Volume_t-1', 'Volume_t0', 'Volume_t1', 'Volume_t2', 'Volume_t3','Volume_t4', 'Volume_t5',
                                               'IND_dummy_0' ,'IND_dummy_1' , 'IND_dummy_2' , 'IND_dummy_3' ,'IND_dummy_4',
                                               'IND_dummy_5' ,'IND_dummy_6' , 'IND_dummy_7' , 'IND_dummy_8' ,'IND_dummy_9', 
                                               'IND_dummy_10','IND_dummy_11', 'IND_dummy_12', 'IND_dummy_13','IND_dummy_14',
                                               'IND_dummy_15', 'IND_dummy_16','IND_dummy_17', 'IND_dummy_18', 'IND_dummy_19', 
                                                'IND_dummy_20','IND_dummy_21', 'IND_dummy_22', 
                                                'Cap', 'First_Quintile','Second_Quintile', 'Third_Quintile', 'Fourth_Quintile','Fifth_Quintile','Quintile_Class'
                                               ]].drop_duplicates().reset_index(drop=True) 
#長度必須等於8705088
len(df_PandQ_Event_dropMultiEvent)

705088


705088

In [41]:
df_PandQ_Event_dropMultiEvent['E_dummy1'] = df_PandQ_Event.groupby(['Code','Date'])['E_dummy1'].sum().tolist()
df_PandQ_Event_dropMultiEvent['E_dummy2'] = df_PandQ_Event.groupby(['Code','Date'])['E_dummy2'].sum().tolist()
df_PandQ_Event_dropMultiEvent['E_dummy3'] = df_PandQ_Event.groupby(['Code','Date'])['E_dummy3'].sum().tolist()
df_PandQ_Event_dropMultiEvent['E_dummy4'] = df_PandQ_Event.groupby(['Code','Date'])['E_dummy4'].sum().tolist()
df_PandQ_Event_dropMultiEvent['JumpHigh'] = np.where((df_PandQ_Event_dropMultiEvent['Open_t+1'].values  - df_PandQ_Event_dropMultiEvent['High'].values)>0,1,0 )
df_PandQ_Event_dropMultiEvent['JumpLow'] = np.where((df_PandQ_Event_dropMultiEvent['Open_t+1'].values  - df_PandQ_Event_dropMultiEvent['Low'].values)<0,1,0 )

df_PandQ_Event_dropMultiEvent['E_dummy1xJumpHigh'] = df_PandQ_Event_dropMultiEvent['E_dummy1'].values*df_PandQ_Event_dropMultiEvent['JumpHigh'].values
df_PandQ_Event_dropMultiEvent['E_dummy1xJumpLow']  = df_PandQ_Event_dropMultiEvent['E_dummy1'].values*df_PandQ_Event_dropMultiEvent['JumpLow'].values
df_PandQ_Event_dropMultiEvent['E_dummy2xJumpHigh'] = df_PandQ_Event_dropMultiEvent['E_dummy2'].values*df_PandQ_Event_dropMultiEvent['JumpHigh'].values
df_PandQ_Event_dropMultiEvent['E_dummy2xJumpLow']  = df_PandQ_Event_dropMultiEvent['E_dummy2'].values*df_PandQ_Event_dropMultiEvent['JumpLow'].values
df_PandQ_Event_dropMultiEvent['E_dummy3xJumpHigh'] = df_PandQ_Event_dropMultiEvent['E_dummy3'].values*df_PandQ_Event_dropMultiEvent['JumpHigh'].values
df_PandQ_Event_dropMultiEvent['E_dummy3xJumpLow']  = df_PandQ_Event_dropMultiEvent['E_dummy3'].values*df_PandQ_Event_dropMultiEvent['JumpLow'].values
df_PandQ_Event_dropMultiEvent['E_dummy4xJumpHigh'] = df_PandQ_Event_dropMultiEvent['E_dummy4'].values*df_PandQ_Event_dropMultiEvent['JumpHigh'].values
df_PandQ_Event_dropMultiEvent['E_dummy4xJumpLow']  = df_PandQ_Event_dropMultiEvent['E_dummy4'].values*df_PandQ_Event_dropMultiEvent['JumpLow'].values

df_PandQ_Event_dropMultiEvent['E1xQ5'] = df_PandQ_Event_dropMultiEvent['E_dummy1'].values * df_PandQ_Event_dropMultiEvent['Fifth_Quintile'].values

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  


In [42]:
df_PandQ_Event_dropMultiEventNa = df_PandQ_Event_dropMultiEvent.dropna().reset_index(drop=True)

In [44]:
df_PandQ_Event_dropMultiEventNa.groupby('Date')['Code'].count()

Date
20140721    660
20140722    659
20140723    662
20140724    661
20140725    660
20140728    663
20140729    665
20140730    665
20140731    667
20140801    667
20140804    665
20140805    669
20140806    668
20140807    671
20140808    669
20140811    667
20140812    663
20140813    660
20140814    662
20140815    662
20140818    658
20140819    659
20140820    660
20140821    662
20140822    661
20140825    659
20140826    657
20140827    658
20140828    653
20140829    654
           ... 
20180604    847
20180605    848
20180606    847
20180607    843
20180608    841
20180611    839
20180612    833
20180613    832
20180614    832
20180615    834
20180619    832
20180620    833
20180621    833
20180622    834
20180625    837
20180626    839
20180627    841
20180628    843
20180629    844
20180702    847
20180703    851
20180704    851
20180705    851
20180706    854
20180709    857
20180710    857
20180711    857
20180712    857
20180713    857
20180716    857
Name: Code, Length:

# 评分系统1 paired t

In [ ]:
import scipy as sp
df_blocktrade=df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['E_dummy1']==1]
df_mortgage =df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['E_dummy2']==1]
df_JumpHigh = df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['JumpHigh']==1]
df_JumpLow = df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['JumpLow']==1]

df_E1xJH = df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['E_dummy1xJumpHigh']==1]
df_E1xJL = df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['E_dummy1xJumpLow']==1]
df_E2xJH = df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['E_dummy2xJumpHigh']==1] 
df_E2xJL = df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['E_dummy2xJumpLow']==1]

df_H1sd =  df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['HighThan1Std' ]==1]
df_L1sd =  df_PandQ_Event_dropMultiEventNa[df_PandQ_Event_dropMultiEventNa['LowThan1Std' ]==1]

In [ ]:
E1_pvalues = sp.stats.ttest_rel(df_blocktrade['MeanF3'],df_blocktrade['MeanP5'])[1]
E2_pvalues = sp.stats.ttest_rel(df_mortgage['MeanF3'],df_mortgage['MeanP5'])[1]
JH_pvalues = sp.stats.ttest_rel(df_JumpHigh['MeanF3'],df_JumpHigh ['MeanP5'])[1]
JL_pvalues = sp.stats.ttest_rel(df_JumpLow['MeanF3'],df_JumpLow['MeanP5'])[1]

E1xJH_pvalues = sp.stats.ttest_rel(df_E1xJH['MeanF3'],df_E1xJH['MeanP5'])[1]
E1xJL_pvalues = sp.stats.ttest_rel(df_E1xJL['MeanF3'],df_E1xJL['MeanP5'])[1]
E2xJH_pvalues = sp.stats.ttest_rel(df_E2xJH['MeanF3'],df_E2xJH['MeanP5'])[1]
E2xJL_pvalues = sp.stats.ttest_rel(df_E2xJL['MeanF3'],df_E2xJL['MeanP5'])[1]

H1sd_pvalues = sp.stats.ttest_rel(df_H1sd['MeanF3'],df_H1sd['MeanP5'])[1]
L1sd_pvalues = sp.stats.ttest_rel(df_L1sd['MeanF3'],df_L1sd['MeanP5'])[1]

# 评分方法二 正负比例

In [ ]:
# 依据paired T 系数决定影响方向
E1_NPType = sp.stats.ttest_rel(df_blocktrade['MeanF3'],df_blocktrade['MeanP5'])[0] 
E2_NPType = sp.stats.ttest_rel(df_mortgage['MeanF3'],df_mortgage['MeanP5'])[0]
JH_NPType = sp.stats.ttest_rel(df_JumpHigh['MeanF3'],df_JumpHigh ['MeanP5'])[0]
JL_NPType = sp.stats.ttest_rel(df_JumpLow['MeanF3'],df_JumpLow['MeanP5'])[0]

E1xJH_NPType = sp.stats.ttest_rel(df_E1xJH['MeanF3'],df_E1xJH['MeanP5'])[0]
E1xJL_NPType = sp.stats.ttest_rel(df_E1xJL['MeanF3'],df_E1xJL['MeanP5'])[0]
E2xJH_NPType = sp.stats.ttest_rel(df_E2xJH['MeanF3'],df_E2xJH['MeanP5'])[0]
E2xJL_NPType = sp.stats.ttest_rel(df_E2xJL['MeanF3'],df_E2xJL['MeanP5'])[0]

H1sd_NPType = sp.stats.ttest_rel(df_H1sd['MeanF3'],df_H1sd['MeanP5'])[0]
L1sd_NPType = sp.stats.ttest_rel(df_L1sd['MeanF3'],df_L1sd['MeanP5'])[0]
NPTypeList = [E1_NPType,E2_NPType,JH_NPType,JL_NPType ,E1xJH_NPType,E1xJL_NPType,E2xJH_NPType,E2xJL_NPType,H1sd_NPType,L1sd_NPType]
NPTypeList_one = np.where(np.array(NPTypeList)>0,1,0)

In [ ]:
def PNrate(data,NPType):
    data = data[(data<np.percentile(data,95)) & (data>np.percentile(data,5))]#去除極值
    data_posi = len(data[data>0])
    data_nega = len(data[data<=0])
    if NPType == 1:
        result =  data_posi/len(data)
    else:
        result =  data_nega/len(data)
    return result

In [ ]:
E1_PNrate = PNrate(df_blocktrade['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[0])
E2_PNrate = PNrate(df_mortgage['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[1])
JH_PNrate = PNrate(df_JumpHigh['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[2])
JL_PNrate = PNrate(df_JumpLow['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[3])

E1xJH_PNrate = PNrate(df_E1xJH['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[4])
E1xJL_PNrate = PNrate(df_E1xJL['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[5])
E2xJH_PNrate = PNrate(df_E2xJH['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[6])
E2xJL_PNrate = PNrate(df_E2xJL['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[7])

H1sd_PNrate = PNrate(df_H1sd['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[8])
L1sd_PNrate = PNrate(df_L1sd['MeanF3lessMeanP5%'].values,NPType=NPTypeList_one[9])

In [120]:
NPTypeList_one

array([1, 0, 1, 0, 1, 1, 1, 0, 1, 0])

# 评分系统三

In [45]:
def countsampleN(data) :
    df_jumpHigh = pd.DataFrame()
    df_jumpLow=pd.DataFrame()

    df_random=pd.DataFrame()
    ##４等分　隨機　大宗、質押　高開　低開
    data_E1_count = len(data[data.E_dummy1 == 1])
    data_E2_count = len(data[data.E_dummy2 == 1])
    data_JH_count = len(data[data.JumpHigh==1])
    data_JL_count = len(data[data.JumpLow==1])
    data_nan_count = len(data[(data.E_dummy1==0)&(data.E_dummy2==0)&(data.JumpHigh==0)&(data.JumpLow==0)])
    
    random_count = np.where(data_E1_count+data_E2_count < 10 ,10,data_E1_count+data_E2_count)
    random_count_jumpH = np.where(random_count>=data_JH_count,data_JH_count,random_count )
    random_count_jumpL = np.where(random_count>=data_JL_count,data_JL_count,random_count )
    random_count_nan  = np.where(random_count>=data_nan_count,data_nan_count,random_count )
    
    #random_count_jumpH = np.where( random_count_jumpH ==0 ,10,random_count_jumpH  )
    #random_count_jumpL = np.where( random_count_jumpL ==0 ,10,random_count_jumpL )   
    
    df_E = data[(data.E_dummy1==1)|(data.E_dummy2==1)]
    if random_count_jumpH ==0:
        df_jumpHigh = data[data.JumpHigh == 1]
    else:
        df_jumpHigh = data[data.JumpHigh == 1].sample(random_count_jumpH)
   
    if random_count_jumpL ==0:
        df_jumpLow = data[data.JumpLow == 1]
    else:
        df_jumpLow = data[data.JumpLow == 1].sample(random_count_jumpL)

    df_nan =data[(data.E_dummy1==0)&(data.E_dummy2==0)&(data.JumpHigh==0)&(data.JumpLow==0)]
    if random_count_nan   ==0:
        df_random = df_nan
    else:
        df_random = df_nan.sample(random_count_nan)


    data = pd.concat([df_random,df_E,df_jumpHigh,df_jumpLow] ).drop_duplicates().reset_index(drop=True)
    return [len(df_E),len(df_jumpHigh),len(df_jumpLow),len(df_random),len(data)]

In [46]:
pd.DataFrame(df_PandQ_Event_dropMultiEventNa.groupby('Date').apply(lambda x : countsampleN(x)))

,0
Date,
20140721,"[0, 10, 10, 10, 30]"
20140722,"[0, 10, 4, 10, 24]"
20140723,"[0, 9, 10, 10, 29]"
20140724,"[0, 10, 10, 10, 30]"
20140725,"[0, 10, 5, 10, 25]"
20140728,"[14, 14, 3, 14, 45]"
20140729,"[23, 24, 16, 24, 84]"
20140730,"[10, 11, 11, 11, 41]"
20140731,"[15, 15, 15, 15, 59]"


In [47]:
def random_efficiency_rank(data):
    df_jumpHigh = pd.DataFrame()
    df_jumpLow=pd.DataFrame()

    df_random=pd.DataFrame()
    ##４等分　隨機　大宗、質押　高開　低開
    data_E1_count = len(data[data.E_dummy1 == 1])
    data_E2_count = len(data[data.E_dummy2 == 1])
    data_JH_count = len(data[data.JumpHigh==1])
    data_JL_count = len(data[data.JumpLow==1])
    data_nan_count = len(data[(data.E_dummy1==0)&(data.E_dummy2==0)&(data.JumpHigh==0)&(data.JumpLow==0)])
    
    random_count = np.where(data_E1_count+data_E2_count < 10 ,10,data_E1_count+data_E2_count)
    random_count_jumpH = np.where(random_count>=data_JH_count,data_JH_count,random_count )
    random_count_jumpL = np.where(random_count>=data_JL_count,data_JL_count,random_count )
    random_count_nan  = np.where(random_count>=data_nan_count,data_nan_count,random_count )
    
    #random_count_jumpH = np.where( random_count_jumpH ==0 ,10,random_count_jumpH  )
    #random_count_jumpL = np.where( random_count_jumpL ==0 ,10,random_count_jumpL )   
    
    df_E = data[(data.E_dummy1==1)|(data.E_dummy2==1)]
    if random_count_jumpH ==0:
        df_jumpHigh = data[data.JumpHigh == 1]
    else:
        df_jumpHigh = data[data.JumpHigh == 1].sample(random_count_jumpH)
   
    if random_count_jumpL ==0:
        df_jumpLow = data[data.JumpLow == 1]
    else:
        df_jumpLow = data[data.JumpLow == 1].sample(random_count_jumpL)

    df_nan =data[(data.E_dummy1==0)&(data.E_dummy2==0)&(data.JumpHigh==0)&(data.JumpLow==0)]
    if random_count_nan   ==0:
        df_random = df_nan
    else:
        df_random = df_nan.sample(random_count_nan)


    data = pd.concat([df_random,df_E,df_jumpHigh,df_jumpLow] ).drop_duplicates().reset_index(drop=True)
    y=data['MeanF3lessMeanP5%']
    X=sm.add_constant(data[['E_dummy1',
      'E_dummy2', 'JumpHigh', 'JumpLow',
       'E_dummy1xJumpHigh', 'E_dummy1xJumpLow', 'E_dummy2xJumpHigh',
       'E_dummy2xJumpLow','LowThan1Std','HighThan1Std','First_Quintile','Fifth_Quintile']])
    Weight = data['Cap']
    result = sm.WLS(y,X,weights=Weight).fit()
    return  [ result.pvalues.tolist() , result.tvalues.tolist(), result.params.tolist(),result.f_pvalue ]

In [48]:
df_PandQ_Event_dropMultiEventNa['MeanF3'] = np.mean(df_PandQ_Event_dropMultiEventNa[['Volume_t0', 'Volume_t1', 'Volume_t2',
       'Volume_t3']],axis=1)
df_PandQ_Event_dropMultiEventNa['MeanP5'] = np.mean(df_PandQ_Event_dropMultiEventNa[['Volume_t-5', 'Volume_t-4', 'Volume_t-3',
       'Volume_t-2', 'Volume_t-1']],axis=1)
df_PandQ_Event_dropMultiEventNa['MeanF3lessMeanP5%'] = (df_PandQ_Event_dropMultiEventNa['MeanF3'].values- df_PandQ_Event_dropMultiEventNa['MeanP5'].values)/df_PandQ_Event_dropMultiEventNa['MeanP5'].values

pvalue_list= df_PandQ_Event_dropMultiEventNa.groupby('Date').apply(lambda x : random_efficiency_rank(x))

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


In [60]:
from progressbar import ProgressBar

total = 50

significant_E1 =[]
significant_E2 = []
significant_jumpH = []
significant_jumpL = []
significant_e1jumpH = []
significant_e1jumpL = []
significant_e2jumpH =[]
significant_e2jumpL=[]
significant_HighThan1Std = []
significant_LowThan1Std=[]

E_dummy1_Beta_pnratios=[]
E_dummy2_Beta_pnratios=[]
JumpHigh_Beta_pnratios=[]
JumpLow_Beta_pnratios=[]
E_dummy1xJumpHigh_Beta_pnratios=[]
E_dummy1xJumpLow_Beta_pnratios=[]
E_dummy2xJumpHigh_Beta_pnratios=[]
E_dummy2xJumpLow_Beta_pnratios=[]
HighThan1Std_Beta_pnratios=[]
LowThan1Std_Beta_pnratios=[]

FAMAT_E_dummy1 =[]
FAMAT_E_dummy2 =[]
FAMAT_JumpHigh =[]
FAMAT_JumpLow = []
FAMAT_E_dummy1xJumpHigh=[]
FAMAT_E_dummy1xJumpLow=[]
FAMAT_E_dummy2xJumpHigh=[]
FAMAT_E_dummy2xJumpLow=[]
FAMAT_HighThan1Std=[]
FAMAT_LowThan1Std=[]

p_value_all=[]
pbar = ProgressBar().start()
for i in range(0,50):
    pvalue_list= df_PandQ_Event_dropMultiEventNa.groupby('Date').apply(lambda x : random_efficiency_rank(x))
#pvalue
    E_dummy1_P  =[ x[0][1] for x in pvalue_list ]
    E_dummy2_P  =[ x[0][2] for x in pvalue_list ]


    JumpHigh_P = [ x[0][3] for x in pvalue_list ]
    JumpLow_P  = [ x[0][4] for x in pvalue_list ]

    E_dummy1xJumpHigh_P = [ x[0][5] for x in pvalue_list ]
    E_dummy1xJumpLow_P = [ x[0][6] for x in pvalue_list ]
    E_dummy2xJumpHigh_P = [ x[0][7] for x in pvalue_list ]
    E_dummy2xJumpLow_P = [ x[0][8] for x in pvalue_list ]

    LowThan1Std = [ x[0][9] for x in pvalue_list ]
    HighThan1Std = [ x[0][10] for x in pvalue_list ]
#显著比例

    significant_E1.append(sum(np.where(np.array(E_dummy1_P)<0.05,1,0 ))/df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1'].sum().mean()].count())
 #大宗
    significant_E2.append(sum(np.where(np.array(E_dummy2_P)<0.05,1,0 ))/df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2'].sum().mean()].count())
 #股權

    significant_jumpH.append(  sum(np.where(np.array(JumpHigh_P)<0.05,1,0 ))/len(E_dummy1_P))
    significant_jumpL.append( sum(np.where(np.array(JumpLow_P)<0.05,1,0 ))/len(E_dummy1_P))

    significant_e1jumpH.append( sum(np.where(np.array(E_dummy1xJumpHigh_P)<0.05,1,0 ))/df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1xJumpHigh'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1xJumpHigh'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1xJumpHigh'].sum().mean()].count())
    significant_e2jumpH.append( sum(np.where(np.array(E_dummy2xJumpHigh_P)<0.05,1,0 ))/df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2xJumpHigh'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2xJumpHigh'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2xJumpHigh'].sum().mean()].count())

    significant_e1jumpL.append( sum(np.where(np.array(E_dummy1xJumpLow_P)<0.05,1,0 ))/df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1xJumpLow'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1xJumpLow'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy1xJumpLow'].sum().mean()].count())
    significant_e2jumpL.append( sum(np.where(np.array(E_dummy2xJumpLow_P)<0.05,1,0 ))/df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2xJumpLow'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2xJumpLow'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['E_dummy2xJumpLow'].sum().mean()].count())

    significant_LowThan1Std.append( sum(np.where(np.array(LowThan1Std)<0.05,1,0 ))/df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum().mean()].count())
    significant_HighThan1Std.append( sum(np.where(np.array(HighThan1Std)<0.05,1,0 ))/df_PandQ_Event_dropMultiEventNa.groupby('Date')['HighThan1Std'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['HighThan1Std'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['HighThan1Std'].sum().mean()].count())
#beta

    E_dummy1_Beta  =[ x[2][1] for x in pvalue_list ]
    E_dummy2_Beta  =[ x[2][2] for x in pvalue_list ]

    JumpHigh_Beta = [ x[2][3] for x in pvalue_list ]
    JumpLow_Beta  = [ x[2][4] for x in pvalue_list ]

    E_dummy1xJumpHigh_Beta = [ x[2][5] for x in pvalue_list ]
    E_dummy1xJumpLow_Beta = [ x[2][6] for x in pvalue_list ]
    E_dummy2xJumpHigh_Beta = [ x[2][7] for x in pvalue_list ]
    E_dummy2xJumpLow_Beta = [ x[2][8] for x in pvalue_list ]

    LowThan1Std_Beta = [ x[2][9] for x in pvalue_list ]
    HighThan1Std_Beta = [ x[2][10] for x in pvalue_list ]
#正负比例
    E_dummy1_Beta_pnratios.append( sum(np.where(np.array(E_dummy1_Beta)>0,1,0 ))/ sum(np.where(np.array(E_dummy1_Beta)<0,1,0 )) ) #大宗
    #-E_dummy2_Beta
    E_dummy2_Beta_pnratios.append( sum(np.where(np.array(E_dummy2_Beta)<0,1,0 ))/ sum(np.where(np.array(E_dummy2_Beta)>0,1,0 )) ) #股權

    JumpHigh_Beta_pnratios.append( sum(np.where(np.array(JumpHigh_Beta)>0.00001,1,0 ))/ sum(np.where(np.array(JumpHigh_Beta)<0,1,0 )))
    #-JumpLow
    JumpLow_Beta_pnratios.append( sum(np.where(np.array(JumpLow_Beta)<0,1,0 ))/ sum(np.where(np.array(JumpLow_Beta)>0.00001,1,0 )))

    E_dummy1xJumpHigh_Beta_pnratios.append( sum(np.where(np.array(E_dummy1xJumpHigh_Beta)>0,1,0 ))/sum(np.where(np.array(E_dummy1xJumpHigh_Beta)<0,1,0 )) )
    E_dummy1xJumpLow_Beta_pnratios.append( sum(np.where(np.array(E_dummy2xJumpHigh_Beta)>0,1,0 ))/ sum(np.where(np.array(E_dummy2xJumpHigh_Beta)<0,1,0 )) )

    E_dummy2xJumpHigh_Beta_pnratios.append( sum(np.where(np.array(E_dummy1xJumpLow_Beta)>0,1,0 ))/ sum(np.where(np.array(E_dummy1xJumpLow_Beta)<0,1,0 )) )
    #-E_dummy2xJumpLow
    E_dummy2xJumpLow_Beta_pnratios.append( sum(np.where(np.array(E_dummy2xJumpLow_Beta)<0,1,0 ))/ sum(np.where(np.array(E_dummy2xJumpLow_Beta)>0,1,0 )) )

    LowThan1Std_Beta_pnratios.append( sum(np.where(np.array(LowThan1Std_Beta)>0,1,0 ))/sum(np.where(np.array(LowThan1Std_Beta)<0,1,0 )) )
    #-HighThan1Std
    HighThan1Std_Beta_pnratios.append( sum(np.where(np.array(HighThan1Std_Beta)<0,1,0 ))/sum(np.where(np.array(HighThan1Std_Beta)>0,1,0 ))) 
#評分標準三 FAMAT
    FAMAT_E_dummy1.append( np.mean(E_dummy1_Beta)/(np.std(E_dummy1_Beta)/np.sqrt(len(E_dummy1_Beta))))
    FAMAT_E_dummy2.append( np.mean(E_dummy2_Beta)/(np.std(E_dummy2_Beta)/np.sqrt(len(E_dummy2_Beta))))

    FAMAT_JumpHigh.append( np.mean(JumpHigh_Beta)/(np.std(JumpHigh_Beta)/np.sqrt(len(JumpHigh_Beta))))
    FAMAT_JumpLow.append( np.mean(JumpLow_Beta)/(np.std(JumpLow_Beta)/np.sqrt(len(JumpLow_Beta))))

    FAMAT_E_dummy1xJumpHigh.append( np.mean(E_dummy1xJumpHigh_Beta)/(np.std(E_dummy1xJumpHigh_Beta)/np.sqrt(len(E_dummy1xJumpHigh_Beta))))
    FAMAT_E_dummy1xJumpLow.append( np.mean(E_dummy1xJumpLow_Beta)/(np.std(E_dummy1xJumpLow_Beta)/np.sqrt(len(E_dummy1xJumpLow_Beta))))
    FAMAT_E_dummy2xJumpHigh.append( np.mean(E_dummy2xJumpHigh_Beta)/(np.std(E_dummy2xJumpHigh_Beta)/np.sqrt(len(E_dummy2xJumpHigh_Beta))))
    FAMAT_E_dummy2xJumpLow.append( np.mean(E_dummy2xJumpLow_Beta)/(np.std(E_dummy2xJumpLow_Beta)/np.sqrt(len(E_dummy2xJumpLow_Beta))))

    FAMAT_HighThan1Std.append( np.mean(HighThan1Std_Beta)/(np.std(HighThan1Std_Beta)/np.sqrt(len(HighThan1Std_Beta))))
    FAMAT_LowThan1Std.append( np.mean(LowThan1Std_Beta)/(np.std(LowThan1Std_Beta)/np.sqrt(len(LowThan1Std_Beta))))


    p_value_all.append(np.percentile(pd.DataFrame([ x[2] for x in pvalue_list ]),70))

    pbar.update(int((i/(total-1))*100))
pbar.finish()

C:\Users\user\Anaconda3\lib\site-packages\statsmodels\base\model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
C:\Users\user\Anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1821: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in less
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:70: RuntimeWarning: invalid value encountered in less
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:71: RuntimeWarning:

In [116]:
df_Rank_System = pd.DataFrame()
df_Rank_System['Event&Indicators'] = ['BlockTrade','Mortgage','JumpHigh','JumpLow','BlockTrade_x_JumpHigh','Mortgage_x_JumpHigh',
                                      'BlockTrade_x_JumpLow','Mortgage_x_JumpLow','HighThan1Std','LowThan1Std']
df_Rank_System['Pair_T'] =  [E1_pvalues,E2_pvalues,JH_pvalues,JL_pvalues ,E1xJH_pvalues,E1xJL_pvalues,E2xJH_pvalues,E2xJL_pvalues,H1sd_pvalues,L1sd_pvalues]
df_Rank_System['PN_Rate']= [E1_PNrate,E2_PNrate,JH_PNrate,JL_PNrate ,E1xJH_PNrate,E1xJL_PNrate,E2xJH_PNrate,E2xJL_PNrate,H1sd_PNrate,L1sd_PNrate]
#因为系统三随机抽取50次 必须取平均
df_Rank_System['Significant_Ratios'] = [np.mean(significant_E1),np.mean(significant_E2),np.mean(significant_jumpH),np.mean(significant_jumpL),
                                        np.mean(significant_e1jumpH),np.mean(significant_e2jumpH),np.mean(significant_e1jumpL),np.mean(significant_e2jumpL),
                                        np.mean(np.array(significant_HighThan1Std)*df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum().mean()].count()/len(df_PandQ_Event_dropMultiEventNa.groupby('Date').count())),
                                        np.mean(np.array(significant_LowThan1Std)*df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum()[df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum()>  df_PandQ_Event_dropMultiEventNa.groupby('Date')['LowThan1Std'].sum().mean()].count()/len(df_PandQ_Event_dropMultiEventNa.groupby('Date').count()))]
df_Rank_System['P/N_Ratios']=[np.mean(E_dummy1_Beta_pnratios),np.mean(E_dummy2_Beta_pnratios),np.mean(JumpHigh_Beta_pnratios),np.mean(JumpLow_Beta_pnratios),
                              np.mean(E_dummy1xJumpHigh_Beta_pnratios),np.mean(E_dummy2xJumpHigh_Beta_pnratios),
                              np.mean(E_dummy1xJumpLow_Beta_pnratios),np.mean(E_dummy2xJumpLow_Beta_pnratios),np.mean(HighThan1Std_Beta_pnratios),np.mean(LowThan1Std_Beta_pnratios)]
df_Rank_System['FAMA_Tratios'] = [np.mean(FAMAT_E_dummy1),np.mean(FAMAT_E_dummy2),np.mean(FAMAT_JumpHigh),np.mean(FAMAT_JumpLow),
                                  np.mean(FAMAT_E_dummy1xJumpHigh),np.mean(FAMAT_E_dummy2xJumpHigh),np.mean(FAMAT_E_dummy1xJumpLow),np.mean(FAMAT_E_dummy2xJumpLow),
                                  np.mean(FAMAT_LowThan1Std),np.mean(FAMAT_HighThan1Std)]

In [115]:
def significant_ranking(data,Type):
    if Type == 'Pair_T':
        listbound = [0.01,0.025,0.05]
        result = np.where(np.array(data)<=listbound[0],3,0)
        result = np.where((np.array(data)>listbound[0])&(np.array(data)<=listbound[1]),2,result)
        result = np.where((np.array(data)>listbound[1])&(np.array(data)<=listbound[2]),1,result)
        result = np.where(np.array(data)>listbound[2],0,result)
    if Type == 'PN_Rate':
        listbound = [0.5,0.6,0.65,0.7]
        result = np.where(np.array(data)<listbound[0],0,0)
        result = np.where((np.array(data)>=listbound[0])&(np.array(data)<listbound[1]),1,result)
        result = np.where((np.array(data)>=listbound[1])&(np.array(data)<listbound[2]),2,result)
        result = np.where((np.array(data)>=listbound[2])&(np.array(data)<listbound[3]),3,result)
        result = np.where(np.array(data)>=listbound[3],4,result)
    if Type == 'Significant_Ratios':
        listbound = [0.5,0.6,0.65,0.7]
        result = np.where(np.array(data)<listbound[0],0,0)
        result = np.where((np.array(data)>=listbound[0])&(np.array(data)<listbound[1]),1,result)
        result = np.where((np.array(data)>=listbound[1])&(np.array(data)<listbound[2]),2,result)
        result = np.where((np.array(data)>=listbound[2])&(np.array(data)<listbound[3]),3,result)
        result = np.where(np.array(data)>=listbound[3],4,result)
    if Type == 'P/N_Ratios':
        listbound = [1.5,1.875,2.33]
        result = np.where(np.array(data)<listbound[0],0,0)
        result = np.where((np.array(data)>=listbound[0])&(np.array(data)<listbound[1]),1,result)
        result = np.where((np.array(data)>=listbound[1])&(np.array(data)<listbound[2]),2,result)
        result = np.where(np.array(data)>=listbound[2],3,result)
    if Type == 'FAMA_Tratios':
        listbound = [1.28,1.645,2]
        result = np.where(abs(np.array(data))<listbound[0],0,0)
        result = np.where((abs(np.array(data))>=listbound[0])&(abs(np.array(data))<listbound[1]),1,result)
        result = np.where((abs(np.array(data))>=listbound[1])&(abs(np.array(data))<listbound[2]),2,result)
        result = np.where(abs(np.array(data))>=listbound[2],3,result)
    return result

In [117]:
df_Rank_System_Rank = pd.DataFrame()
df_Rank_System_Rank['Event&Indicators'] = ['BlockTrade','Mortgage','JumpHigh','JumpLow','BlockTrade_x_JumpHigh','Mortgage_x_JumpHigh',
                                      'BlockTrade_x_JumpLow','Mortgage_x_JumpLow','HighThan1Std','LowThan1Std']

df_Rank_System_Rank['Pair_T'] = significant_ranking(df_Rank_System['Pair_T'],Type='Pair_T')
df_Rank_System_Rank['PN_Rate']= significant_ranking(df_Rank_System['PN_Rate'] ,Type='PN_Rate')
df_Rank_System_Rank['Significant_Ratios'] = significant_ranking(df_Rank_System['Significant_Ratios'] ,Type='Significant_Ratios')
df_Rank_System_Rank['P/N_Ratios'] = significant_ranking(df_Rank_System['P/N_Ratios'] ,Type='P/N_Ratios')
df_Rank_System_Rank['FAMAT_Ratios'] = significant_ranking(df_Rank_System['FAMA_Tratios'] ,Type='FAMA_Tratios')
df_Rank_System_Rank['Sum_Rating'] = np.sum(df_Rank_System_Rank[['Pair_T','PN_Rate','Significant_Ratios','P/N_Ratios','FAMAT_Ratios']],axis=1)



In [119]:
df_Rank_System_Rank.to_csv('Ranking_System_all.csv')

In [121]:
df_Rank_System_Rank

,Event&Indicators,Pair_T,PN_Rate,Significant_Ratios,P/N_Ratios,FAMAT_Ratios,Sum_Rating
0,BlockTrade,3,1,0,0,0,4
1,Mortgage,0,1,0,0,0,1
2,JumpHigh,3,3,0,3,3,12
3,JumpLow,3,2,0,0,3,8
4,BlockTrade_x_JumpHigh,3,3,0,0,0,6
5,Mortgage_x_JumpHigh,3,0,0,0,0,3
6,BlockTrade_x_JumpLow,3,2,0,0,0,5
7,Mortgage_x_JumpLow,0,2,0,0,0,2
8,HighThan1Std,3,4,0,3,0,10
9,LowThan1Std,3,4,0,0,3,10
